In [2]:
using LinearAlgebra
using JuMP
using GLPK
using Plots
pyplot()

Plots.PyPlotBackend()

In [3]:
# Problema do sapateiro (Q4-L1) versão 1
model = Model()
set_optimizer(model, GLPK.Optimizer)

# Define as variáveis
@variable(model, x>=0)
@variable(model, y>=0)

# Define as restrições
@constraint(model, x+y<=10)
@constraint(model, 12x+5y<=78)

# Define a função objetivo
@objective(model, Max, 30x+20y)

# Roda a otimização
optimize!(model)

# Exibe os resultados
println("Valor ótimo:   ", objective_value(model))
println("Solução ótima: ", (value.(x), value.(y)))

Valor ótimo:   240.0
Solução ótima: (4.0, 6.0)


In [2]:
# Problema do sapateiro (Q4-L1) versão 2
model = Model()
set_optimizer(model, GLPK.Optimizer)

# Define as variáveis
@variable(model, x>=0)
@variable(model, y>=0)

# Define as restrições
@constraint(model, x/6+y/5<=10)
@constraint(model, 2x+y<=78)

# Define a função objetivo
@objective(model, Max, 5x+4y)

# Roda a otimização
optimize!(model)

# Exibe os resultados
println("Valor ótimo:   ", objective_value(model))
println("Solução ótima: ", (value.(x), value.(y)))

Valor ótimo:   240.0
Solução ótima: (24.0, 29.999999999999996)


In [5]:
# Problema do combustível para avião (Q5-L1)
model = Model()
set_optimizer(model, GLPK.Optimizer)

# Define as variáveis
@variable(model, x>=0)
@variable(model, y>=0)

# Define as restrições
@constraint(model, 0.25x<=500)
@constraint(model, 0.25x+0.5y<=200)
@constraint(model, 0.5x+0.5y<=200)

# Define a função objetivo
@objective(model, Max, 20x+30y)

# Roda a otimização
optimize!(model)

# Exibe os resultados
println("Valor ótimo:   ", objective_value(model))
println("Solução ótima: ", (value.(x), value.(y)))

Valor ótimo:   12000.0
Solução ótima: (0.0, 400.0)


In [207]:
function PLI(max_min, i, j, weights, A, b, C, d, E, f)
    model = Model(GLPK.Optimizer) # Cria modelo
    p = length(weights)

    # Define variáveis
    @variable(model, x[1:i, 1:j] >= 0)
    x = reshape(x', p, 1)

    # Define restrições
    for l in 1:length(f)
        @constraint(model, (E*x)[l] >= f[l]) # Restrições de desigualdade >=
    end
    for m in 1:length(b)
        @constraint(model, (A*x)[m] == b[m]) # Restrições de igualdade
    end
    for n in 1:length(d)
        @constraint(model, (C*x)[n] <= d[n]) # Restrições de desigualdade <=
    end

    # Define função objetivo
    if max_min == 1
        @objective(model, Max, sum(weights[k] * x[k] for k in 1:p)) # Problema de maximização
    else
        @objective(model, Min, sum(weights[k] * x[k] for k in 1:p)) # Problema de minimização
    end
    
    # Roda a otimização
    optimize!(model)
    
    # Exibe resultados
    println("Função objetivo: ", objective_function(model))
    println()
    println("Valor ótimo: ", objective_value(model))
    println()
    println("Solução ótima:")
    for k in 1:p
        println(x[k], " = ", value(x[k]))
    end
end

PLI (generic function with 3 methods)

In [210]:
# Problema da companhia energética (Q2-A1)
A = [1 0 0 0 0 0 0 0 0;0 1 0 0 1 0 0 1 0;0 0 1 0 0 1 0 0 1]
b = [20;10;30]
C = [1 1 1 0 0 0 0 0 0;0 0 0 0 1 1 0 0 0;0 0 0 0 0 0 0 1 1]
d = [50;20;50]
weights = [50 90 80 0 60 50 0 30 40]
i = 3
j = 3

PLI(0, i, j, weights, 0, 0, C, d, A, b)

Função objetivo: 50 x[1,1] + 90 x[1,2] + 80 x[1,3] + 60 x[2,2] + 50 x[2,3] + 30 x[3,2] + 40 x[3,3]

Valor ótimo: 2500.0

Solução ótima:
x[1,1] = 20.0
x[1,2] = 0.0
x[1,3] = 0.0
x[2,1] = 0.0
x[2,2] = 0.0
x[2,3] = 0.0
x[3,1] = 0.0
x[3,2] = 10.0
x[3,3] = 30.0


In [213]:
# Problema das usinas da fábrica (Q1-L1)
E = [1 0 0 1 0 0;0 1 0 0 1 0;0 0 1 0 0 1]
f = [100;200;300]
C = [1 1 1 0 0 0;0 0 0 1 1 1]
d = [400;300]
weights = [1 1.5 3.5 2 1 2]
i = 2
j = 3

PLI(0, i, j, weights, 0, 0, C, d, E, f)

Função objetivo: x[1,1] + 1.5 x[1,2] + 3.5 x[1,3] + 2 x[2,1] + x[2,2] + 2 x[2,3]

Valor ótimo: 1000.0

Solução ótima:
x[1,1] = 100.0
x[1,2] = 200.0
x[1,3] = 0.0
x[2,1] = 0.0
x[2,2] = 0.0
x[2,3] = 300.0


In [217]:
# Problema da usina química (Q2-L1)
C = [1 1;2 3;0 1]
d = [8;19;4]
weights = [7 9]
i = 2
j = 1

PLI(1, i, j, weights, 0, 0, C, d, 0, 0)

Função objetivo: 7 x[1,1] + 9 x[2,1]

Valor ótimo: 62.0

Solução ótima:
x[1,1] = 5.0
x[2,1] = 3.0


In [214]:
# Problema das usinas de carro (Q3-L1)
E = [1 0 0 1 0 0;0 1 0 0 1 0;0 0 1 0 0 1]
f = [5;4;3]
C = [1 1 1 0 0 0;0 0 0 1 1 1]
d = [6;6]
weights = [38 27 48 37 58 45]
i = 2
j = 3

PLI(0, i, j, weights, 0, 0, C, d, E, f)

Função objetivo: 38 x[1,1] + 27 x[1,2] + 48 x[1,3] + 37 x[2,1] + 58 x[2,2] + 45 x[2,3]

Valor ótimo: 430.0

Solução ótima:
x[1,1] = 2.0
x[1,2] = 4.0
x[1,3] = 0.0
x[2,1] = 3.0
x[2,2] = 0.0
x[2,3] = 3.0


In [215]:
# Problema do sapateiro (Q4-L1) versão 1
C = [1 1;12 5]
d = [10;78]
weights = [30 20]
i = 2
j = 1

PLI(1, i, j, weights, 0, 0, C, d, 0, 0)

Função objetivo: 30 x[1,1] + 20 x[2,1]

Valor ótimo: 240.0

Solução ótima:
x[1,1] = 4.0
x[2,1] = 6.0


In [216]:
# Problema do sapateiro (Q4-L1) versão 2
C = [1/6 1/5;2 1]
d = [10;78]
weights = [5 4]
i = 2
j = 1

PLI(1, i, j, weights, 0, 0, C, d, 0, 0)

Função objetivo: 5 x[1,1] + 4 x[2,1]

Valor ótimo: 240.0

Solução ótima:
x[1,1] = 24.0
x[2,1] = 29.999999999999996


In [212]:
# Problema do combustível para avião (Q5-L1)
C = [0.25 0;0.25 0.5;0.5 0.5]
d = [500;200;200]
weights = [20 30]
i = 2
j = 1

PLI(1, i, j, weights, 0, 0, C, d, 0, 0)

Função objetivo: 20 x[1,1] + 30 x[2,1]

Valor ótimo: 12000.0

Solução ótima:
x[1,1] = 0.0
x[2,1] = 400.0


In [6]:
# Problema da fábrica de petróleo (Q6-L1)

# Criar o modelo com o solver GLPK
model = Model(GLPK.Optimizer)

# Variáveis de decisão
@variable(model, x[1:4, 1:3] >= 0)  # Quantidade de cada petróleo em cada diesel

d1 = x[1,1] + x[2,1] + x[3,1] + x[4,1]  # Diesel A
d2 = x[1,2] + x[2,2] + x[3,2] + x[4,2]  # Diesel B
d3 = x[1,3] + x[2,3] + x[3,3] + x[4,3]  # Diesel C
d = [d1, d2, d3]

# Função Objetivo: Maximizar o lucro total
@objective(model, Max, 5.5d[1] + 4.5d[2] + 3.5d[3] - (3*(x[1,1] + x[1,2] + x[1,3]) + 6*(x[2,1] + x[2,2] + x[2,3]) + 4*(x[3,1] + x[3,2] + x[3,3]) + 5*(x[4,1] + x[4,2] + x[4,3])))

# Restrições
@constraint(model, x[1,1] + x[1,2] + x[1,3] <= 3000)  # Estoque petróleo 1
@constraint(model, x[2,1] + x[2,2] + x[2,3] <= 2000)  # Estoque petróleo 2
@constraint(model, x[3,1] + x[3,2] + x[3,3] <= 4000)  # Estoque petróleo 3
@constraint(model, x[4,1] + x[4,2] + x[4,3] <= 1000)  # Estoque petróleo 4
@constraint(model, x[1,1] <= 0.3d[1])                # Proporção em diesel A
@constraint(model, x[3,1] <= 0.5d[1])                # Proporção em diesel A
@constraint(model, x[2,1] >= 0.4d[1])                # Proporção em diesel A
@constraint(model, x[2,2] >= 0.1d[2])                # Proporção em diesel B
@constraint(model, x[1,2] <= 0.5d[2])                # Proporção em diesel B
@constraint(model, x[1,3] <= 0.7d[3])                # Proporção em diesel C

# Resolver o modelo
optimize!(model)

# Exibir os resultados
println("Lucro máximo: ", objective_value(model))
println("Quantidades de petróleo para diesel A, B, C:")
for j in 1:3
    println("Diesel ", j, ":")
    for i in 1:4
        println("  Petróleo ", i, ": ", value(x[i,j]))
    end
    println("  Diesel total ", j, ": ", value(d[j]))
end


Lucro máximo: 7166.666666666666
Quantidades de petróleo para diesel A, B, C:
Diesel 1:
  Petróleo 1: 1099.9999999999998
  Petróleo 2: 1466.6666666666665
  Petróleo 3: 1099.9999999999995
  Petróleo 4: 0.0
  Diesel total 1: 3666.6666666666656
Diesel 2:
  Petróleo 1: 1900.0
  Petróleo 2: 533.3333333333334
  Petróleo 3: 2900.0000000000005
  Petróleo 4: 0.0
  Diesel total 2: 5333.333333333334
Diesel 3:
  Petróleo 1: 0.0
  Petróleo 2: 0.0
  Petróleo 3: 0.0
  Petróleo 4: 0.0
  Diesel total 3: 0.0


In [6]:
# Problema da mistura de líquidos (Q1-A1)

# Criar o modelo
model = Model(GLPK.Optimizer)

# Definição das variáveis
@variable(model, x[1:4, 1:3] >= 0)  # Quantidades de líquidos nas misturas
@variable(model, l[1:4] >= 0)       # Quantidades de líquidos vendidos diretamente
@variable(model, p >= 0)            # Quantidade do produto P

# Função objetivo: Maximizar receita - custo
@objective(model, Max, 
    6 * sum(l) + 22 * p + 11 * (sum(x[:, 1])-p/3) + 15 * sum(x[:, 2]) + 14 * (sum(x[:, 3])-2/3 * p) - 
    (5.5 * (sum(x[1, :] .+ l[1])) + 4.5 * (sum(x[2, :] .+ l[2])) + 
    7.5 * (sum(x[3, :] .+ l[3])) + 11.25 * (sum(x[4, :] .+ l[4])))
)

# Restrições de estoque
@constraint(model, sum(x[1, :]) + l[1] <= 8000)
@constraint(model, sum(x[2, :]) + l[2] <= 4250)
@constraint(model, sum(x[3, :]) + l[3] <= 16000)
@constraint(model, sum(x[4, :]) + l[4] <= 2000)

# Composição das misturas
@constraint(model, x[1,1] == 0.3 * sum(x[:,1]))  # Mistura E
@constraint(model, x[2,1] >= 0.1 * sum(x[:,1]))  # Mistura E
@constraint(model, x[3,1] == 0.4 * sum(x[:,1]))  # Mistura E
@constraint(model, x[4,1] <= 0.05 * sum(x[:,1])) # Mistura E

@constraint(model, x[1,2] >= 0.25 * sum(x[:,2])) # Mistura F
@constraint(model, x[2,2] <= 0.2 * sum(x[:,2]))  # Mistura F
@constraint(model, x[3,2] == 0.2 * sum(x[:,2]))  # Mistura F
@constraint(model, x[4,2] >= 0.1 * sum(x[:,2]))  # Mistura F

@constraint(model, x[1,3] == 0.2 * sum(x[:,3]))  # Mistura G
@constraint(model, x[2,3] >= 0.15 * sum(x[:,3])) # Mistura G
@constraint(model, x[3,3] == 0.4 * sum(x[:,3]))  # Mistura G
@constraint(model, x[4,3] <= 0.2 * sum(x[:,3]))  # Mistura G

# Produção mínima das misturas
@constraint(model, sum(x[:,1]) >= 400)
@constraint(model, sum(x[:,2]) >= 800)
@constraint(model, sum(x[:,3]) >= 200)

# Relação para produção de P
@constraint(model, sum(x[:,1]) - p/3 >= 0)
@constraint(model, sum(x[:,3]) - 2/3 * p >= 0)

# Resolver o modelo
optimize!(model)

# Exibir os resultados
println("Lucro máximo: ", objective_value(model))
println("Quantidades de líquidos vendidos diretamente e em misturas:")
for i in 1:4
    println("Líquido ", i, ": vendido diretamente = ", value(l[i]), ", em misturas = ", value(sum(x[i,:])))
end
println("Quantidade de produto P: ", value(p))


Lucro máximo: 290821.42857142864
Quantidades de líquidos vendidos diretamente e em misturas:
Líquido 1: vendido diretamente = 0.0, em misturas = 8000.0
Líquido 2: vendido diretamente = 0.0, em misturas = 4250.0
Líquido 3: vendido diretamente = 0.0, em misturas = 7392.857142857144
Líquido 4: vendido diretamente = 0.0, em misturas = 2000.0
Quantidade de produto P: 15321.428571428576


In [17]:
# Planificação da expansão da produção (Q3-L2)

model = Model(GLPK.Optimizer)

# Dados do problema
m = 4  # Número de máquinas
n = 3  # Número de produtos
T = 100  # Máximo de horas de revisão
p = [400, 400, 400]  # Custo por unidade não vendida
c = [2.5, 3.75, 5.0, 3.0]  # Custo por hora adicional
h = [500, 500, 500, 500]  # Horas regulares de funcionamento
u = [2000, 2000, 3000, 3000]  # Cota superior de horas
t = [0.08, 0.04, 0.03, 0.01]  # Taxa de revisão por hora

a = [0.6 0.6 0.9 0.8; 
     0.1 0.9 0.6 0.8; 
     0.05 0.2 0.5 0.8]  # Matriz de eficiência de produção

g = [2.6 3.4 3.4 2.5;
     1.5 2.4 2.0 3.6;
     4.0 3.8 3.5 3.2]  # Matriz de custo por hora

d = [1800, 600, 3000]  # Demanda dos produtos

# Variáveis
@variable(model, h_ij[1:n, 1:m] >= 0)  # Horas de produção de produto i em máquina j
@variable(model, e[1:m] >= 0)          # Horas extras adquiridas para máquina j
@variable(model, s[1:n] >= 0)          # Variáveis auxiliares para linearização

# Função Objetivo
@objective(model, Min, sum(c[j] * e[j] for j in 1:m) + 
                       sum(g[i,j] * h_ij[i,j] for i in 1:n, j in 1:m) + 
                       sum(p[i] * s[i] for i in 1:n))


# Restrições de linearização
for i in 1:n
    @constraint(model, s[i] >= sum(a[i, j] * h_ij[i, j] for j in 1:m) - d[i])
end

# Denais restrições
for j in 1:m
    @constraint(model, sum(h_ij[:, j]) + e[j] <= u[j])  # Horas totais disponíveis por máquina
    @constraint(model, sum(t[j] * (sum(h_ij[:, j]) + e[j])) <= T)  # Limite de tempo de revisão
end

for i in 1:n
    @constraint(model, sum(a[i,j] * h_ij[i,j] for j in 1:m) >= d[i])  # Demanda do produto i
end

# Resolver o modelo
optimize!(model)

# Exibir os resultados
println("Custo mínimo: ", objective_value(model))
for j in 1:m
    println("Máquina ", j, ":")
    println("  Horas extras adquiridas = ", value(e[j]))
    for i in 1:n
        println("  Horas produzindo produto ", i, ": ", value(h_ij[i,j]))
    end
end


Custo mínimo: 22300.0
Máquina 1:
  Horas extras adquiridas = 0.0
  Horas produzindo produto 1: 300.0
  Horas produzindo produto 2: 0.0
  Horas produzindo produto 3: 0.0
Máquina 2:
  Horas extras adquiridas = 0.0
  Horas produzindo produto 1: 0.0
  Horas produzindo produto 2: 666.6666666666666
  Horas produzindo produto 3: 0.0
Máquina 3:
  Horas extras adquiridas = 0.0
  Horas produzindo produto 1: 1800.0
  Horas produzindo produto 2: 0.0
  Horas produzindo produto 3: 1200.0
Máquina 4:
  Horas extras adquiridas = 0.0
  Horas produzindo produto 1: 0.0
  Horas produzindo produto 2: 0.0
  Horas produzindo produto 3: 3000.0
